# Dostosowywanie modeli Open AI

Ten notatnik opiera się na aktualnych wytycznych zawartych w dokumentacji [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) od Open AI.

Dostosowywanie poprawia wydajność modeli bazowych dla Twojej aplikacji poprzez ponowne trenowanie ich z dodatkowymi danymi i kontekstem istotnym dla konkretnego przypadku użycia lub scenariusza. Zauważ, że techniki inżynierii promptów, takie jak _few shot learning_ i _retrieval augmented generation_, pozwalają na wzbogacenie domyślnego promptu o odpowiednie dane w celu poprawy jakości. Jednak te podejścia są ograniczone przez maksymalny rozmiar okna tokenów docelowego modelu bazowego.

Dzięki dostosowywaniu skutecznie ponownie trenujemy sam model z wymaganymi danymi (co pozwala nam użyć znacznie więcej przykładów niż mieści się w maksymalnym oknie tokenów) - i wdrażamy _niestandardową_ wersję modelu, która nie wymaga już podawania przykładów podczas inferencji. To nie tylko poprawia skuteczność naszego projektowania promptów (mamy większą elastyczność w wykorzystaniu okna tokenów na inne rzeczy), ale potencjalnie także obniża koszty (poprzez zmniejszenie liczby tokenów, które musimy wysłać do modelu podczas inferencji).

Dostosowywanie ma 4 kroki:
1. Przygotuj dane treningowe i prześlij je.
1. Uruchom zadanie treningowe, aby uzyskać dostosowany model.
1. Oceń dostosowany model i iteruj dla jakości.
1. Wdróż dostosowany model do inferencji, gdy będziesz zadowolony.

Zauważ, że nie wszystkie modele bazowe obsługują dostosowywanie - [sprawdź dokumentację OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) dla najnowszych informacji. Możesz także dostosować wcześniej dostosowany model. W tym samouczku użyjemy `gpt-35-turbo` jako naszego docelowego modelu bazowego do dostosowywania.

---


### Krok 1.1: Przygotuj swój zestaw danych

Zbudujmy chatbota, który pomoże Ci zrozumieć układ okresowy pierwiastków, odpowiadając na pytania o pierwiastek za pomocą limeryku. W _tym_ prostym samouczku stworzymy tylko zestaw danych do trenowania modelu z kilkoma przykładowymi odpowiedziami pokazującymi oczekiwany format danych. W rzeczywistym zastosowaniu musiałbyś stworzyć zestaw danych z wieloma przykładami. Możesz także użyć otwartego zestawu danych (dla swojej dziedziny zastosowań), jeśli taki istnieje, i przekształcić go do użytku w fine-tuningu.

Ponieważ skupiamy się na `gpt-35-turbo` i szukamy odpowiedzi jednokrotnej (chat completion), możemy stworzyć przykłady używając [tego sugerowanego formatu](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) odzwierciedlającego wymagania OpenAI dla chat completion. Jeśli spodziewasz się wielokrotnych tur konwersacji, użyłbyś [formatu przykładów wieloturnowych](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), który zawiera parametr `weight` sygnalizujący, które wiadomości powinny być użyte (lub nie) w procesie fine-tuningu.

Dla naszego samouczka użyjemy prostszego formatu jednokrotnego. Dane są w formacie [jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) z 1 rekordem na linię, każdy reprezentowany jako obiekt w formacie JSON. Poniższy fragment pokazuje 2 rekordy jako przykład - zobacz [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) dla pełnego zestawu przykładowego (10 przykładów), którego użyjemy w naszym samouczku fine-tuningu. **Uwaga:** Każdy rekord _musi_ być zdefiniowany w jednej linii (nie dzielony na wiele linii jak to zwykle bywa w sformatowanym pliku JSON)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

W rzeczywistym zastosowaniu będziesz potrzebować znacznie większego zestawu przykładów dla dobrych wyników - kompromis będzie między jakością odpowiedzi a czasem/kosztami fine-tuningu. Używamy małego zestawu, aby szybko zakończyć fine-tuning i zilustrować proces. Zobacz [ten przykład z OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) dla bardziej złożonego samouczka fine-tuningu.


---

### Krok 1.2 Prześlij swój zestaw danych

Prześlij dane za pomocą Files API [zgodnie z opisem tutaj](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Zwróć uwagę, że aby uruchomić ten kod, musisz najpierw wykonać następujące kroki:
 - Zainstalować pakiet `openai` dla Pythona (upewnij się, że używasz wersji >=0.28.0 dla najnowszych funkcji)
 - Ustawić zmienną środowiskową `OPENAI_API_KEY` na swój klucz API OpenAI
Aby dowiedzieć się więcej, zobacz [Przewodnik konfiguracji](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) udostępniony na potrzeby kursu.

Teraz uruchom kod, aby utworzyć plik do przesłania z lokalnego pliku JSONL.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Krok 2.1: Utwórz zadanie dostrajania za pomocą SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Krok 2.2: Sprawdź status zadania

Oto kilka rzeczy, które możesz zrobić za pomocą API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Wyświetl ostatnie n zadań fine-tuningu
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Pobierz szczegóły konkretnego zadania fine-tuningu
- `client.fine_tuning.jobs.cancel(<job_id>)` - Anuluj zadanie fine-tuningu
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Wyświetl do n zdarzeń z zadania
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Pierwszym krokiem procesu jest _walidacja pliku treningowego_, aby upewnić się, że dane mają odpowiedni format.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### Krok 2.3: Śledź zdarzenia, aby monitorować postępy


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Krok 2.4: Sprawdź status w panelu OpenAI


Możesz również sprawdzić status, odwiedzając stronę OpenAI i przeglądając sekcję _Fine-tuning_ na platformie. Pokaże Ci to status bieżącego zadania, a także pozwoli śledzić historię wcześniejszych uruchomień zadań. Na tym zrzucie ekranu widać, że poprzednie uruchomienie zakończyło się niepowodzeniem, a drugie uruchomienie powiodło się. Dla kontekstu, zdarzyło się to, gdy pierwsze uruchomienie używało pliku JSON z niepoprawnie sformatowanymi rekordami – po naprawieniu drugie uruchomienie zakończyło się sukcesem i udostępniło model do użytku.

![Fine-tuning job status](../../../../../translated_images/pl/fine-tuned-model-status.563271727bf7bfba.png)


Możesz również zobaczyć komunikaty statusu i metryki, przewijając dalej w wizualnym panelu, jak pokazano:

| Komunikaty | Metryki |
|:---|:---|
| ![Messages](../../../../../translated_images/pl/fine-tuned-messages-panel.4ed0c2da5ea1313b.png) |  ![Metrics](../../../../../translated_images/pl/fine-tuned-metrics-panel.700d7e4995a65229.png)|


---

### Krok 3.1: Pobierz ID i przetestuj dostrojony model w kodzie


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Krok 3.2: Załaduj i przetestuj dostrojony model w Playground

Teraz możesz przetestować dostrojony model na dwa sposoby. Po pierwsze, możesz odwiedzić Playground i użyć rozwijanego menu Models, aby wybrać swój nowo dostrojony model z dostępnych opcji. Drugą opcją jest użycie opcji „Playground” pokazanej w panelu Fine-tuning (zobacz zrzut ekranu powyżej), która uruchamia następujący _porównawczy_ widok pokazujący wersje modelu bazowego i dostrojonego obok siebie do szybkiej oceny.

![Fine-tuning job status](../../../../../translated_images/pl/fine-tuned-playground-compare.56e06f0ad8922016.png)

Wystarczy wypełnić kontekst systemowy użyty w danych treningowych i podać swoje pytanie testowe. Zauważysz, że obie strony są aktualizowane tym samym kontekstem i pytaniem. Uruchom porównanie, a zobaczysz różnicę w wynikach między nimi. _Zwróć uwagę, jak model dostrojony generuje odpowiedź w formacie, który podałeś w swoich przykładach, podczas gdy model bazowy po prostu podąża za podpowiedzią systemową_.

![Fine-tuning job status](../../../../../translated_images/pl/fine-tuned-playground-launch.5a26495c983c6350.png)

Zauważysz, że porównanie pokazuje również liczbę tokenów dla każdego modelu oraz czas potrzebny na inferencję. **Ten konkretny przykład jest prosty i ma na celu pokazanie procesu, ale nie odzwierciedla rzeczywistego zestawu danych ani scenariusza**. Możesz zauważyć, że oba przykłady pokazują tę samą liczbę tokenów (kontekst systemowy i podpowiedź użytkownika są identyczne), przy czym model dostrojony potrzebuje więcej czasu na inferencję (model niestandardowy).

W rzeczywistych scenariuszach nie będziesz używać takiego przykładu, lecz dostrajanie na podstawie prawdziwych danych (np. katalog produktów dla obsługi klienta), gdzie jakość odpowiedzi będzie znacznie bardziej widoczna. W _tym_ kontekście uzyskanie równoważnej jakości odpowiedzi za pomocą modelu bazowego będzie wymagało bardziej zaawansowanego inżynierii podpowiedzi, co zwiększy zużycie tokenów i potencjalnie czas przetwarzania inferencji. _Aby to wypróbować, sprawdź przykłady dostrajania w OpenAI Cookbook, aby zacząć._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Zastrzeżenie**:  
Niniejszy dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mimo że dokładamy starań, aby tłumaczenie było jak najbardziej precyzyjne, prosimy mieć na uwadze, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w języku źródłowym powinien być uznawany za źródło autorytatywne. W przypadku informacji krytycznych zalecane jest skorzystanie z profesjonalnego tłumaczenia wykonanego przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z korzystania z tego tłumaczenia.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
